# 패키지 설치

In [ ]:
import copy
import pathlib
import random
from functools import reduce
from collections import defaultdict
import json
import pandas as pd
import numpy as np
from numpy import random

import matplotlib.pyplot as plt
import seaborn as sns

# 공간연산한 결과 데이터 표현하기 위한 패키지 
import folium
from folium import plugins
import shapely
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from IPython.display import display
from tqdm.notebook import tqdm
import sklearn.cluster
from geoband import API # API 부르기 
import pydeck as pdk
import os
from tqdm import notebook
import tqdm

import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="./data/NanumGothic-Regular.ttf").get_name()
rc('font', family=font_name)

from shapely.geometry import Polygon, Point
from numpy import random

#최적화 solver
import time
from mip import Model, xsum, maximize, BINARY  

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.model_selection import train_test_split
import pysal
import libpysal
from libpysal.weights import Queen, Rook, KNN, DistanceBand, Kernel
from esda.getisord import G
from esda.moran import Moran
from spreg import OLS, ML_Lag, ML_Error, GM_Lag, GM_Error, TSLS

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
# 노트북에서 만든 data_list load
file_df = pd.read_csv("./data/data_list.csv")

# order_num type change int to str
file_df["order_num"] = file_df["order_num"].apply(lambda x: str(x))

# 동적변수 데이터 할당
for idx in tqdm.tqdm(range(len(file_df) - 2)):
#     print(idx)
    file_name = ".".join(file_df.iloc[idx, :])
    globals()[f"df_{idx + 1}"] = gpd.read_file("./data/" + file_name)

100%|██████████| 32/32 [03:11<00:00,  5.98s/it]


# TOD 분석

In [ ]:
# 노선수 구하기
df_lst = [df_2, df_3, df_4, df_5, df_6]
df = pd.concat(df_lst)
df_route = df.groupby(["정류소ID", "노선번호"])["전체 승차 건수"].agg("sum").reset_index()
df_route_count = df_route.groupby("정류소ID")["노선번호"].agg("count").reset_index()
df_route_count.rename({"정류소ID":"정류장ID", "노선번호":"노선수"}, axis = 1, inplace=True)
df_route_count["정류장ID"] = df_route_count["정류장ID"].astype("int")

In [4]:
build_df = gpd.GeoDataFrame(pd.read_csv("./data/build_df.csv"))
bus_buffer = gpd.GeoDataFrame(pd.read_csv("./data/bus_buffer.csv"))
bus_point = gpd.GeoDataFrame(pd.read_csv("./data/bus_point.csv"))
df_100 = gpd.GeoDataFrame(pd.read_csv("./data/df_100.csv"))
df_500 = gpd.GeoDataFrame(pd.read_csv("./data/df_500.csv"))
build_polygon = gpd.GeoDataFrame(pd.read_csv("./data/500_build_polygon.csv"))
df_100_to_500 = gpd.GeoDataFrame(pd.read_csv("./data/df_100_to_500.csv"))

In [5]:
def str_to_geo(x):
    a = shapely.wkt.loads(x)
    return a

build_df["geometry"] = build_df["geometry"].apply(lambda x: str_to_geo(x))
build_df["center"] = build_df["center"].apply(lambda x: str_to_geo(x))
bus_buffer["geometry"] = bus_buffer["geometry"].apply(lambda x: str_to_geo(x))
bus_point["geometry"] = bus_point["geometry"].apply(lambda x: str_to_geo(x))
df_100["geometry"] = df_100["geometry"].apply(lambda x: str_to_geo(x))
df_100["gid_center"] = df_100["gid_center"].apply(lambda x: str_to_geo(x))
build_polygon["geometry"] = build_polygon["geometry"].apply(lambda x: str_to_geo(x))

승차건수가 0인 정류장ID drop

- df_500

In [6]:
drop_index = df_500[df_500["전체 승차 건수"] == 0]["정류장ID"].index.values
df_500 = df_500.drop(drop_index).reset_index(drop = True)

drop_index = df_500[df_500["전체 승차 건수"] == 0].index.values
df_500.drop(drop_index, inplace = True)
df_500.reset_index(drop = True, inplace = True)

df_500 = pd.merge(df_500, df_route_count, on = "정류장ID")

In [7]:
# geometry poin 생성
geo = gpd.points_from_xy(df_500.lon, df_500.lat)
df_500["geometry"] = geo

# 결측치 채우기
df_500 = df_500.fillna(value=0)
df_100 = df_100.fillna(0)
df_100_to_500 = df_100_to_500.fillna(0)

In [8]:
# y변수
df_500_eda = df_500.loc[:, ["정류장ID","전체 승차 건수", "초승 건수", "환승 건수", "전체 하차 건수", "노선수"]]
cols = df_500_eda.iloc[:, 1:-1].columns.values

In [9]:
coor_1m = (1/88.74/1000)
crit_area = df_500["geometry"][0].buffer(coor_1m * 500).area

In [10]:
build_col = df_100_to_500.iloc[:, -8:].columns.values

In [11]:
# 밀도 변수 crit_area 나누기
# 실제 폴리곤 내 밀도 구하기
def division_build(df, build_col, crit_area):
    for col in build_col:
        df[col] = df[col] / crit_area
        
build_lst = [df_100, df_500, df_100_to_500]
for df in build_lst:
    division_build(df, build_col, crit_area)

In [12]:
# 승하차건수 변수
df_500["승하차건수"] = df_500["전체 승차 건수"] + df_500["전체 하차 건수"]

In [13]:
# 총 인구
df_500["population"] = df_500["old"] + df_500["worker"] + df_500["youth"]
df_100["population"] = df_100["old"] + df_100["worker"] + df_100["youth"]
df_100_to_500["population"] = df_100_to_500["old"] + df_100_to_500["worker"] + df_100_to_500["youth"]

In [14]:
# 직주비
df_500["직주비"] = df_500["worker"] / df_500["population"]
df_100["직주비"] = df_100["worker"] / df_100["population"]
df_100_to_500["직주비"] = df_100_to_500["worker"] / df_100_to_500["population"]

In [15]:
# 결측치 채우기
df_500.fillna(0, inplace = True)
df_100.fillna(0, inplace = True)
df_100_to_500.fillna(0, inplace = True)

In [16]:
# x 변수 추출
col_1 = df_500.iloc[:, 21:34].columns.values
col_2 = df_500.iloc[:, -8:].drop("노선수", axis = 1).columns.values
x_col = np.concatenate([col_1, col_2])
x_col = np.delete(x_col, np.where(x_col == "승하차건수"))

# y변수
y_col = ["전체 승차 건수", "초승 건수", "환승 건수", "전체 하차 건수", "승하차건수"]

정규화 log1p

In [17]:
for df in build_lst:
    df[x_col] = np.log1p(df[x_col])
    
# y 정규화
df_500[y_col] = np.log1p(df_500[y_col])
df_500[x_col] = np.log1p(df_500[x_col])
df_100[x_col] = np.log1p(df_100[x_col])
df_100_to_500[x_col] = np.log1p(df_100_to_500[x_col])

In [18]:
df_500_eda = copy.deepcopy(df_500[y_col])

# 군집분석

In [5]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score

In [20]:
# 밀도 기반 클러스터링
dbscan = DBSCAN(eps = 0.8, min_samples=8, metric="euclidean")
dbscan_labels = dbscan.fit_predict(df_500[x_col])

df_500["label"] = dbscan_labels
df_500["label"].value_counts()

 0    1085
 1      18
-1      11
Name: label, dtype: int64

In [ ]:
# 군집분석 실루엣계수
def score_plot(df, col):
    score_lst = []
    for i in range(2, 6):
        kmeans = KMeans(n_clusters=i, init="k-means++", max_iter=300)
        kmeans.fit(df[col])
        df["label"] = kmeans.labels_

        score_samples = silhouette_samples(df[col], df["label"])
        df["silhouette_coeff"] = score_samples

        # 모든 데이터의 평균 실루엣 계수 값을 구함.
        average_score = silhouette_score(df[col], df["label"])
        score_lst.append(average_score)
        print(df["label"].value_counts())
        print("평균 실루엣 값 : %f" % (average_score))
    sns.lineplot(x = range(2, 6), y = score_lst, markers=True)
    plt.xlabel("군집 수")
    plt.ylabel("실루엣 계수")
    plt.ylim(0,1)
    plt.grid(True, axis = "y", alpha = 0.5, linestyle = "--")
    
score_plot(df_500, x_col)

In [22]:
# 실루엣계수가 높은 2로 설정
kmeans = KMeans(n_clusters=2, init="k-means++", max_iter=300)
kmeans.fit(df_500[x_col])
df_500["label"] = kmeans.labels_

# 각 cluster에 속하는 index 추출
idx_0 = df_500[df_500["label"] == 0].index.values
idx_1 = df_500[df_500["label"] == 1].index.values

# 각 cluster별 df indexing
df_500_0 = copy.deepcopy(df_500.iloc[idx_0, :].reset_index(drop = True))
df_500_1 = copy.deepcopy(df_500.iloc[idx_1, :].reset_index(drop = True))

In [ ]:
# 군집별 통계적 차이 검증
from scipy import stats

col_lst = []
for col in x_col:
    from scipy import stats
    stats, pvalue = stats.ttest_ind(df_500_0[col], df_500_1[col])
    if pvalue <= 0.05:
        print(f"{col}는 통계적으로 유의미한 차이가 있다.")
        col_lst.append(col)
        
for col in y_col:
    from scipy import stats
    stats, pvalue = stats.ttest_ind(df_500_0[col], df_500_1[col])
    if pvalue <= 0.05:
        print(f"{col}는 통계적으로 유의미한 차이가 있다.")
        col_lst.append(col)

In [24]:
# 1클러스터는 대부분의 변수에서 높은 값을 보였다.
# 공업시설이 많은 곳은 인구수가 적어 승하차건수가 작다.

In [25]:
# 유의한 차이가 없는 변수 확인
for col in x_col:
    if col not in col_lst:
        print(f"{col}은 통계적으로 유의미한 차이가 없다")
        
for col in y_col:
    if col not in col_lst:
        print(f"{col}은 통계적으로 유의미한 차이가 없다")

공공시설은 통계적으로 유의미한 차이가 없다
공업시설은 통계적으로 유의미한 차이가 없다


In [ ]:
# 각 군집 변수 별 평균 차이 그래프
plt.figure(figsize = (8, 5))


sns.lineplot(x = df_500_0[x_col].describe().loc["mean", :"업무시설"].index,
            y = df_500_0[x_col].describe().loc["mean", :"업무시설"].values, color = "red", label = "0번 군집")
sns.lineplot(x = df_500_1[x_col].describe().loc["mean", :"업무시설"].index,
            y = df_500_1[x_col].describe().loc["mean", :"업무시설"].values, color = "blue", label = "1번 군집")
plt.grid(True, axis = "y", alpha = 0.5, linestyle = "--")
plt.xlabel("변수")
plt.ylabel("평균 값")

In [ ]:
# 각 군집 변수 별 평균 차이 그래프
plt.figure(figsize = (8, 5))

sns.lineplot(x = df_500_0[x_col].describe().loc["mean", "업무시설":].index,
            y = df_500_0[x_col].describe().loc["mean", "업무시설":].values, color = "red", label = "0번 군집")
sns.lineplot(x = df_500_1[x_col].describe().loc["mean", "업무시설":].index,
            y = df_500_1[x_col].describe().loc["mean", "업무시설":].values, color = "blue", label = "1번 군집")
plt.grid(True, axis = "y", alpha = 0.5, linestyle = "--")

plt.xlabel("변수",)
plt.xticks(rotation = 45)
plt.ylabel("평균 값")

In [ ]:
# 상관분석
plt.figure(figsize = (10,5))
sns.heatmap(df_500[x_col].corr(), annot=True)
plt.xticks(rotation = 90)
plt.yticks(rotation = 45)

In [ ]:
# kmean clustering gid 격자 
score_plot(df_100, x_col)

In [30]:
# kmeans = KMeans(n_clusters=2, init="k-means++", max_iter=300)
label = kmeans.fit_predict(df_100[x_col])
df_100["label"] = label

In [ ]:
# 각 cluster에 속하는 index 추출
idx_0 = df_100[df_100["label"] == 0].index.values
idx_1 = df_100[df_100["label"] == 1].index.values

# 각 cluster별 df indexing
df_100_0 = copy.deepcopy(df_100.iloc[idx_0, :].reset_index(drop = True))
df_100_1 = copy.deepcopy(df_100.iloc[idx_1, :].reset_index(drop = True))

# 군집별 통계적 차이 검증
from scipy import stats

col_lst = []
for col in x_col:
    from scipy import stats
    stats, pvalue = stats.ttest_ind(df_100_0[col], df_100_1[col])
    if pvalue <= 0.05:
        print(f"{col}는 통계적으로 유의미한 차이가 있다.")
        col_lst.append(col)

In [32]:
# 공업시설과 지하철역수를 제외한 나머지 변수들에서 0군집이 높은 평균값을 가졌다
for col in x_col:
    if col not in col_lst:
        print(col)

공공시설
교육연구시설
문화체육시설


In [ ]:
plt.figure(figsize = (8, 5))


sns.lineplot(x = df_100_0[col_lst].describe().loc["mean", :"업무시설"].index,
            y = df_100_0[col_lst].describe().loc["mean", :"업무시설"].values, color = "red", label = "0번 군집")
sns.lineplot(x = df_100_1[col_lst].describe().loc["mean", :"업무시설"].index,
            y = df_100_1[col_lst].describe().loc["mean", :"업무시설"].values, color = "blue", label = "1번 군집")
plt.grid(True, axis = "y", alpha = 0.5, linestyle = "--")
plt.xlabel("변수")
plt.ylabel("평균 값")

In [ ]:
plt.figure(figsize = (8, 5))


sns.lineplot(x = df_100_0[col_lst].describe().loc["mean", "업무시설":].index,
            y = df_100_0[col_lst].describe().loc["mean", "업무시설":].values, color = "red", label = "0번 군집")
sns.lineplot(x = df_100_1[col_lst].describe().loc["mean", "업무시설":].index,
            y = df_100_1[col_lst].describe().loc["mean", "업무시설":].values, color = "blue", label = "1번 군집")
plt.grid(True, axis = "y", alpha = 0.5, linestyle = "--")

plt.xlabel("변수",)
plt.xticks(rotation = 45)
plt.ylabel("평균 값")

# 군집분석 시각화

In [ ]:
# 수원시 군집 지도 시각화
style0 = {'fillColor': '#228B22', 'color': 'black', "opacity":"0.3", "fillOpacity":"0"}
style1 = {'fillColor': '#228B22', 'color': 'blue', "fillOpacity":"0","opacity":"0.3"}
style2 = {'fillColor': '#00FFFFFF', 'color': 'red', "fillOpacity":"0","opacity":"0.3"}
style3 = {'fillColor': '#00FFFFFF', 'color': 'red', "opacity":"0.5"}
m = folium.Map([37,127], zoom_start=7)

df_100_0.crs = "epsg:4326"
df_100_1.crs = "epsg:4326"
# df_500_0.crs = "epsg:4326"
# bus_buffer.to_crs = {'init' :'epsg:4326'}

m = folium.Map(max_bounds = True,
               zoom_start=12,
               location=[37.277896,127.016544])



# for idx in range(len(df_500)):
#     folium.vector_layers.Polygon(
#         locations = build_df.loc[idx, "geometry"]
#     ).add_to(m)

folium.GeoJson("./data/31.수원시_행정경계(읍면동).geojson", name="geojson", style_function = lambda x: style0).add_to(m)
folium.GeoJson(data = df_100_0.loc[:, "geometry"], style_function = lambda x:style1).add_to(m)
folium.GeoJson(data = df_100_1.loc[:, "geometry"], style_function = lambda x:style2).add_to(m)
m

In [ ]:
# 수원시 지도 시각화 df_100_0 값이 적은것
style0 = {'fillColor': '#228B22', 'color': 'black', "opacity":"0.3", "fillOpacity":"0"}
style1 = {'fillColor': '#228B22', 'color': 'blue', "fillOpacity":"0","opacity":"0.3"}
style2 = {'fillColor': '#00FFFFFF', 'color': 'red', "fillOpacity":"0","opacity":"0.3"}
style3 = {'fillColor': '#00FFFFFF', 'color': 'red', "opacity":"0.5"}
m = folium.Map([37,127], zoom_start=7)

df_100_0.crs = "epsg:4326"
df_100_1.crs = "epsg:4326"
df_500_0.crs = "epsg:4326"
# bus_buffer.to_crs = {'init' :'epsg:4326'}

m = folium.Map(max_bounds = True,
               zoom_start=12,
               location=[37.277896,127.016544])



# for idx in range(len(df_500)):
#     folium.vector_layers.Polygon(
#         locations = build_df.loc[idx, "geometry"]
#     ).add_to(m)

folium.GeoJson("./data/31.수원시_행정경계(읍면동).geojson", name="geojson", style_function = lambda x: style0).add_to(m)
folium.GeoJson(data = df_100_0.loc[:, "geometry"], style_function = lambda x:style1).add_to(m)
folium.GeoJson(data = df_500_0.loc[:, "geometry"], style_function = lambda x:style1).add_to(m)
# folium.GeoJson(data = df_100_1.loc[:, "geometry"], style_function = lambda x:style2).add_to(m)
m

In [ ]:
# 수원시 지도 시각화 df_100_1 값 높은 거
style0 = {'fillColor': '#228B22', 'color': 'black', "opacity":"0.3", "fillOpacity":"0"}
style1 = {'fillColor': '#228B22', 'color': 'blue', "fillOpacity":"0","opacity":"0.3"}
style2 = {'fillColor': '#00FFFFFF', 'color': 'red', "fillOpacity":"0","opacity":"0.3"}
style3 = {'fillColor': '#00FFFFFF', 'color': 'red', "opacity":"0.5"}
m = folium.Map([37,127], zoom_start=7)

df_100_0.crs = "epsg:4326"
df_100_1.crs = "epsg:4326"
df_500_1.crs = "epsg:4326"
# bus_buffer.to_crs = {'init' :'epsg:4326'}

m = folium.Map(max_bounds = True,
               zoom_start=12,
               location=[37.277896,127.016544])



# for idx in range(len(df_500)):
#     folium.vector_layers.Polygon(
#         locations = build_df.loc[idx, "geometry"]
#     ).add_to(m)

folium.GeoJson("./data/31.수원시_행정경계(읍면동).geojson", name="geojson", style_function = lambda x: style0).add_to(m)
# folium.GeoJson(data = df_100_0.loc[:, "geometry"], style_function = lambda x:style1).add_to(m)
folium.GeoJson(data = df_100_1.loc[:, "geometry"], style_function = lambda x:style2).add_to(m)
folium.GeoJson(data = df_500_1.loc[:, "geometry"], style_function = lambda x:style2).add_to(m)
m

In [ ]:
# df_100_to_500
df_100_to_500.fillna(0, inplace =True)
score_plot(df_100_to_500, x_col)

In [ ]:
label = kmeans.fit_predict(df_100_to_500[x_col])
df_100_to_500["label"] = label
# 각 cluster에 속하는 index 추출
idx_0 = df_100_to_500[df_100_to_500["label"] == 0].index.values
idx_1 = df_100_to_500[df_100_to_500["label"] == 1].index.values

# 각 cluster별 df indexing
df_100_to_500_0 = copy.deepcopy(df_100_to_500.iloc[idx_0, :].reset_index(drop = True))
df_100_to_500_1 = copy.deepcopy(df_100_to_500.iloc[idx_1, :].reset_index(drop = True))

# 군집별 통계적 차이 검증
from scipy import stats

col_lst = []
for col in x_col:
    from scipy import stats
    stats, pvalue = stats.ttest_ind(df_100_to_500_0[col], df_100_to_500_1[col])
    if pvalue <= 0.05:
        print(f"{col}는 통계적으로 유의미한 차이가 있다.")
        col_lst.append(col)

In [40]:
# distance
for col in x_col:
    if col not in col_lst:
        print(f"{col}은 통계적으로 유의미한 차이가 없다")

In [ ]:
plt.figure(figsize = (8, 5))


sns.lineplot(x = df_100_to_500_0[col_lst].describe().loc["mean", :"업무시설"].index,
            y = df_100_to_500_0[col_lst].describe().loc["mean", :"업무시설"].values, color = "red", label = "0번 군집")
sns.lineplot(x = df_100_to_500_1[col_lst].describe().loc["mean", :"업무시설"].index,
            y = df_100_to_500_1[col_lst].describe().loc["mean", :"업무시설"].values, color = "blue", label = "1번 군집")
plt.grid(True, axis = "y", alpha = 0.5, linestyle = "--")
plt.xlabel("변수")
plt.ylabel("평균 값")

In [ ]:
plt.figure(figsize = (8, 5))


sns.lineplot(x = df_100_to_500_0[col_lst].describe().loc["mean", "업무시설":].index,
            y = df_100_to_500_0[col_lst].describe().loc["mean", "업무시설":].values, color = "red", label = "0번 군집")
sns.lineplot(x = df_100_to_500_1[col_lst].describe().loc["mean", "업무시설":].index,
            y = df_100_to_500_1[col_lst].describe().loc["mean", "업무시설":].values, color = "blue", label = "1번 군집")
plt.grid(True, axis = "y", alpha = 0.5, linestyle = "--")

plt.xlabel("변수",)
plt.xticks(rotation = 45)
plt.ylabel("평균 값")

In [43]:
# 군집변수 추가 공업시설이 높은 값은 1번군집
df_100_to_500["label"] = 0
df_100_to_500.loc[idx_1, "label"] = 0
df_100_to_500.loc[idx_0, "label"] = 1

In [44]:
# x변수에 군집변수 추가
x_col = np.append(x_col, ["label"])

In [ ]:
df_100["rold"] = df_100["old"] / df_100["old"].max()

In [ ]:
df_100["rpop"] = df_100["pop"] / df_100["pop"].max()
df_100["rbuild"] = df_100["건물면적"] / df_100["건물면적"].max()

In [ ]:
# pydeck에 시각화 하기 위해서는 polygon을 coordinates 형식으로 바꿔줘야함
def multipolygon_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

df_100['coordinates'] = df_100['geometry'].apply(multipolygon_to_coordinates)
# del df['geometry']

In [ ]:
# pydeck 시각화
# Make layer
layer1 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                  df_100, # 시각화에 쓰일 데이터프레임 
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[227, 255, 227, 80]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 ) 

layer2 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                  df_100, # 시각화에 쓰일 데이터프레임 
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[255*rbuild,200,227, 1000*rbuild]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 ) 

# Set the viewport location 
center = [127.016544,37.277896] # 오산시 중심
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=11
) 

# Render 
r = pdk.Deck(layers=[layer1, layer2], initial_view_state=view_state,
             mapbox_key = "pk.eyJ1IjoiamNsYXJhODExIiwiYSI6ImNrZzF4bWNhdTBpNnEydG54dGpxNDEwajAifQ.XWxOKQ-2HqFBVBYa-XoS-g"
            )

r.to_html()

In [ ]:
style0 = {'fillColor': '#228B22', 'color': 'black', "opacity":"0.3", "fillOpacity":"0"}
style1 = {'fillColor': '#228B22', 'color': 'black', "fillOpacity":"0"}
style2 = {'fillColor': '#00FFFFFF', 'color': 'blue', "opacity":"1"}
style3 = {'fillColor': '#00FFFFFF', 'color': 'red', "opacity":"0.5"}
m = folium.Map([37,127], zoom_start=7)


df.crs = "epsg:4326"
build_polygon.crs = "epsg:4326"
build_df.crs = "epsg:4326"
df_500.crs = "epsg:4326"
df_500_1.crs = "epsg:4326"
bus_buffer.crs = 'epsg:4326'

m = folium.Map(max_bounds = True,
               zoom_start=12,
               location=[37.277896,127.016544])



# for idx in range(len(df_500)):
#     folium.vector_layers.Polygon(
#         locations = build_df.loc[idx, "geometry"]
#     ).add_to(m)



folium.GeoJson("./data/31.수원시_행정경계(읍면동).geojson", name="geojson", style_function = lambda x: style0).add_to(m)
folium.GeoJson(data = bus_buffer["geometry"], style_function = lambda x:style1).add_to(m)
# folium.GeoJson(data = build_polygon.iloc[:5000,:], style_function=lambda x:style2).add_to(m)
folium.GeoJson(data = build_df.iloc[:,:2], style_function=lambda x:style3).add_to(m)
m

In [45]:
road_geo = pd.read_csv("./data/road_transport.csv")

In [46]:
road_geo = gpd.GeoDataFrame(road_geo)
road_geo["geometry"] = road_geo["geometry"].apply(lambda x: str_to_geo(x))

In [ ]:
style0 = {'fillColor': '#228B22', 'color': 'black', "opacity":"0.3", "fillOpacity":"0"}
style1 = {'fillColor': '#228B22', 'color': 'black', "fillOpacity":"0"}
style2 = {'fillColor': '#00FFFFFF', 'color': 'blue', "opacity":"1"}
style3 = {'fillColor': '#00FFFFFF', 'color': 'red', "opacity":"0.5"}
m = folium.Map([37,127], zoom_start=7)


df.crs = "epsg:4326"
build_polygon.crs = "epsg:4326"
build_df.crs = "epsg:4326"
df_500.crs = "epsg:4326"
df_500_1.crs = "epsg:4326"
bus_buffer.crs = 'epsg:4326'
road_geo.crs = "epsg:4326"
df_100.crs = 'epsg:4326'

m = folium.Map(max_bounds = True,
               zoom_start=12,
               location=[37.277896,127.016544])



# for idx in range(len(df_500)):
#     folium.vector_layers.Polygon(
#         locations = build_df.loc[idx, "geometry"]
#     ).add_to(m)



folium.GeoJson("./data/31.수원시_행정경계(읍면동).geojson", name="geojson", style_function = lambda x: style0).add_to(m)
folium.GeoJson(data = df_100["geometry"], style_function = lambda x:style1).add_to(m)
folium.GeoJson(data = road_geo, style_function=lambda x:style3).add_to(m)
folium.GeoJson(data = df_500["geometry"], style_function=lambda x:style3).add_to(m)
m

In [47]:
#df_500.to_csv("./data/df_500_train.csv", index = False)

In [48]:
#df_100_to_500.to_csv("./data/df_100_to_500_test.csv", index = False)

In [ ]:
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

y = df_500["승하차건수"]
x = df_500[x_col]

x_tr, x_test, y_tr, y_test = train_test_split(x, y, test_size = 0.8)

lr_reg = LinearRegression()
sgd_reg = SGDRegressor(max_iter = 300)
rf_reg = RandomForestRegressor()

lr_reg.fit(x_tr, y_tr)
sgd_reg.fit(x_tr, y_tr)
rf_reg.fit(x_tr, y_tr)
lr_pred = lr_reg.predict(x_test)
sgd_pred = sgd_reg.predict(x_test)
rf_pred = rf_reg.predict(x_test)

print("선형회귀 결과 : ", r2_score(y_test, lr_pred))
print("경사하강 결과 : ", r2_score(y_test, sgd_pred))
print("랜덤포레스트 결과 : ", r2_score(y_test, rf_pred))

In [ ]:
# 상관분석
plt.figure(figsize = (10, 5))
sns.heatmap(df_500[x_col].corr(), annot=True)

In [ ]:
# 인구정보 하위 변수는 population 안에 속하는 값이므로 제거
plt.figure(figsize = (10, 5))
sns.heatmap(df_500[["old", "worker", "youth", "population"]].corr(), annot=True)

In [ ]:
# 건물면적, 주거시설 상관성이 높음
# 건물면적의 하위 변수들인 다른 밀도들은 상관성이 낮기 때문에 하위 변수인 주거시설 사용
plt.figure(figsize = (10, 5))
sns.heatmap(df_500[["건물면적", "주거시설"]].corr(), annot=True)

In [ ]:
# 인구정보 변수중 population만 독립변수로 사용
x_col = np.delete(x_col, np.where((x_col == ["old"]) | (x_col == ["worker"]) | (x_col == ["youth"])))

In [ ]:
x_col = np.delete(x_col, np.where(x_col == "건물면적"))

In [ ]:
# 제거 후 상관분석 다시 실행
plt.figure(figsize = (10, 5))
sns.heatmap(df_500[x_col].corr(), annot=True)

In [ ]:
# pop(유동인구) building_area, 직주비, populatoin
# builing_area는 건물밀도로 확인 가능하므로 독립변수에서 제거
# 직주비는 TOD 변수이므로 population 변수 돌립변수에서 제거
plt.figure(figsize = (10, 5))
sns.heatmap(df_500[["pop", "building_area", "직주비", "population"]].corr(), annot=True)

In [ ]:
# 독립변수에서 제거
x_col = np.delete(x_col, np.where(x_col == "building_area"))
x_col = np.delete(x_col, np.where(x_col == "population"))

# 공간회귀분석

In [ ]:
#회귀분석

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.model_selection import train_test_split

#공간회귀분석

import pysal
import libpysal
from libpysal.weights import Queen, Rook, KNN, DistanceBand, Kernel
from esda.getisord import G
from esda.moran import Moran
from spreg import OLS, ML_Lag, ML_Error, GM_Lag, GM_Error, TSLS


#정규성검정
from scipy.stats import shapiro

#다중공선성 체크
from statsmodels.stats.outliers_influence import variance_inflation_factor

다중공선성 체크

In [ ]:
# 다중공선성 확인 "버스정류장수" 제일 높음
vif = pd.DataFrame()
vif["vif_value"] = [variance_inflation_factor(df_500[x_col].values, i) for i in range(len(x_col))]
vif["columns"] = x_col

vif

In [ ]:
# 다중공선성 확인2. "버스정류장수" 제거
vif = pd.DataFrame()
col = np.delete(x_col, np.where(x_col == "버스정류장수"))
vif["vif_value"] = [variance_inflation_factor(df_500[col].values, i) for i in range(len(col))]
vif["columns"] = col

vif

In [ ]:
# 다중공선성 확인3.  "pop" 제거
# 밀도로 유추 가능
vif = pd.DataFrame()
col = np.delete(x_col, np.where((x_col == "버스정류장수") | (x_col == "pop")))
vif["vif_value"] = [variance_inflation_factor(df_500[col].values, i) for i in range(len(col))]
vif["columns"] = col

vif

In [ ]:
# 다중공선성 확인4. "직주비" 제거
# 주거시설로 유츠 가능
vif = pd.DataFrame()
col = np.delete(x_col, np.where((x_col == "버스정류장수") | (x_col == "pop")
                               | (x_col == "직주비")))
vif["vif_value"] = [variance_inflation_factor(df_500[col].values, i) for i in range(len(col))]
vif["columns"] = col

vif

In [ ]:
# 다중공선성 확인5. "distance" 제거
# 구할때 부터 평균으로 들어감
vif = pd.DataFrame()
col = np.delete(x_col, np.where((x_col == "버스정류장수") | (x_col == "pop")
                               | (x_col == "직주비") ))
vif["vif_value"] = [variance_inflation_factor(df_500[col].values, i) for i in range(len(col))]
vif["columns"] = col

vif

In [ ]:
# 다중공선성 확인6. "label" 제거
# 새로 수집한 변수 제거
vif = pd.DataFrame()
col = np.delete(x_col, np.where((x_col == "버스정류장수") | (x_col == "pop")
                               | (x_col == "직주비") | (x_col == "distance")
                               ))
vif["vif_value"] = [variance_inflation_factor(df_500[col].values, i) for i in range(len(col))]
vif["columns"] = col

vif

In [ ]:
# y 절편
df_500 = sm.add_constant(df_500, has_constant="add")

공간회귀분석

In [ ]:
W_Q = Queen.from_dataframe(df_500)
W_D = DistanceBand.from_dataframe(df_500, threshold=0.005, binary = False)

In [ ]:
W_Q.transform = "r"
W_D.transform = "r"

In [ ]:
x = np.array(df_500[x_col])
x_1 = np.array(df_500[col])

y = np.array(df_500[["승하차건수"]])

In [ ]:
ols_Q = OLS(y= y, x= x, w = W_Q, name_x = list(x_col), name_y="승하차건수", white_test=True, spat_diag=True, moran=True)
ols_Q_1 = OLS(y= y, x= x_1, w = W_Q, name_x = list(col), name_y="승하차건수", white_test=True, spat_diag=True, moran=True)

ols_D = OLS(y= y, x= x, w = W_D, name_x = list(x_col), name_y="승하차건수", white_test=True, spat_diag=True, moran=True)
ols_D_1 = OLS(y= y, x= x_1, w = W_D, name_x = list(col), name_y="승하차건수", white_test=True, spat_diag=True, moran=True)

In [ ]:
print(ols_Q.summary)

In [ ]:
print(ols_Q_1.summary)

In [ ]:
# 정규성 검증
# 모든 변수가 정규성을 가정하지 않는다.
for idx in col:
    stat, pvalue = shapiro(df_500[col])
    if pvalue >= 0.05:
        print(idx)
    else:
        print(idx, "변수는 정규성을 가정하지 않는다")

In [ ]:
# sem_ml 모형
mlerror_q = ML_Error(y=y, x=x_1, w=W_Q, method='full', name_x=list(col), name_y=["승하차건수"])
gmerror = GM_Error(x = x_1, name_x= list(col), y = y, name_y = '승하차건수', w = W_Q)

In [ ]:
print(mlerror_q.summary)

In [ ]:
print(gmerror.summary)

In [ ]:
# 유의한 변수 선택 '공공시설', '교육연구시설', '업무시설', '지하철역수'

new_col = np.delete(col, np.where((col == "공공시설") | (col == '교육연구시설') 
                                  | (col == '업무시설') | (col == '지하철역수') | (col == "label")
                                 | (col == 'distance')))

In [ ]:
# 유의한 변수로 돌리기
x_1 = np.array(df_500[new_col])
    
y = np.array(df_500[["승하차건수"]])

In [ ]:
mlerror_q = ML_Error(y=y, x=x_1, w=W_Q, method='full', name_x=list(new_col), name_y=["승하차건수"])
gmerror = GM_Error(x = x_1, name_x= list(new_col), y = y, name_y = '승하차건수', w = W_Q)

In [ ]:
print(mlerror_q.summary)

In [ ]:
print(gmerror.summary)

In [ ]:
new_col

In [ ]:
pd.DataFrame({"x_col":new_col}).to_csv("./data/x_col.csv", index = False)